# Generating N-Gaussian data

In the previous tutorial, we learned how to generate **Gaussian** data using the **GaussianDistribution** class. For this tutorial we'll be looking at generating _N_ Gaussian models simultaneously.

To do this, we can use the **generators** module from `pyMultiFit` library,

In [1]:
from pymultifit.generators import multi_gaussian

Let's say we want to generate 3 Gaussian distributions with $\mu_i = [-3, 1, 5]$ and $\sigma_i = [0.1, 0.5, 1]$, we can pass